In [3]:
#Taken from Stackoverflow


import os
from collections import defaultdict
import re
from pandas import Series,DataFrame
import pandas as pd
import pickle
import numpy as np

def striprtf(text):
   pattern = re.compile(r"(\\cell\b)|\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)", re.I)
   # control words which specify a "destionation".
   destinations = frozenset((
      'aftncn','aftnsep','aftnsepc','annotation','atnauthor','atndate','atnicn','atnid',
      'atnparent','atnref','atntime','atrfend','atrfstart','author','background',
      'bkmkend','bkmkstart','blipuid','buptim','category','colorschememapping',
      'colortbl','comment','company','creatim','datafield','datastore','defchp','defpap',
      'do','doccomm','docvar','dptxbxtext','ebcend','ebcstart','factoidname','falt',
      'fchars','ffdeftext','ffentrymcr','ffexitmcr','ffformat','ffhelptext','ffl',
      'ffname','ffstattext','field','file','filetbl','fldinst','fldrslt','fldtype',
      'fname','fontemb','fontfile','fonttbl','footer','footerf','footerl','footerr',
      'footnote','formfield','ftncn','ftnsep','ftnsepc','g','generator','gridtbl',
      'header','headerf','headerl','headerr','hl','hlfr','hlinkbase','hlloc','hlsrc',
      'hsv','htmltag','info','keycode','keywords','latentstyles','lchars','levelnumbers',
      'leveltext','lfolevel','linkval','list','listlevel','listname','listoverride',
      'listoverridetable','listpicture','liststylename','listtable','listtext',
      'lsdlockedexcept','macc','maccPr','mailmerge','maln','malnScr','manager','margPr',
      'mbar','mbarPr','mbaseJc','mbegChr','mborderBox','mborderBoxPr','mbox','mboxPr',
      'mchr','mcount','mctrlPr','md','mdeg','mdegHide','mden','mdiff','mdPr','me',
      'mendChr','meqArr','meqArrPr','mf','mfName','mfPr','mfunc','mfuncPr','mgroupChr',
      'mgroupChrPr','mgrow','mhideBot','mhideLeft','mhideRight','mhideTop','mhtmltag',
      'mlim','mlimloc','mlimlow','mlimlowPr','mlimupp','mlimuppPr','mm','mmaddfieldname',
      'mmath','mmathPict','mmathPr','mmaxdist','mmc','mmcJc','mmconnectstr',
      'mmconnectstrdata','mmcPr','mmcs','mmdatasource','mmheadersource','mmmailsubject',
      'mmodso','mmodsofilter','mmodsofldmpdata','mmodsomappedname','mmodsoname',
      'mmodsorecipdata','mmodsosort','mmodsosrc','mmodsotable','mmodsoudl',
      'mmodsoudldata','mmodsouniquetag','mmPr','mmquery','mmr','mnary','mnaryPr',
      'mnoBreak','mnum','mobjDist','moMath','moMathPara','moMathParaPr','mopEmu',
      'mphant','mphantPr','mplcHide','mpos','mr','mrad','mradPr','mrPr','msepChr',
      'mshow','mshp','msPre','msPrePr','msSub','msSubPr','msSubSup','msSubSupPr','msSup',
      'msSupPr','mstrikeBLTR','mstrikeH','mstrikeTLBR','mstrikeV','msub','msubHide',
      'msup','msupHide','mtransp','mtype','mvertJc','mvfmf','mvfml','mvtof','mvtol',
      'mzeroAsc','mzeroDesc','mzeroWid','nesttableprops','nextfile','nonesttables',
      'objalias','objclass','objdata','object','objname','objsect','objtime','oldcprops',
      'oldpprops','oldsprops','oldtprops','oleclsid','operator','panose','password',
      'passwordhash','pgp','pgptbl','picprop','pict','pn','pnseclvl','pntext','pntxta',
      'pntxtb','printim','private','propname','protend','protstart','protusertbl','pxe',
      'result','revtbl','revtim','rsidtbl','rxe','shp','shpgrp','shpinst',
      'shppict','shprslt','shptxt','sn','sp','staticval','stylesheet','subject','sv',
      'svb','tc','template','themedata','title','txe','ud','upr','userprops',
      'wgrffmtfilter','windowcaption','writereservation','writereservhash','xe','xform',
      'xmlattrname','xmlattrvalue','xmlclose','xmlname','xmlnstbl',
      'xmlopen',
   ))
   # Translation of some special characters.
   specialchars = {
      'par': '\n',
      'sect': '\n\n',
      'page': '\n\n',
      'line': '\n',
      'tab': '\t',
      'emdash': '\u2014',
      'endash': '\u2013',
      'emspace': '\u2003',
      'enspace': '\u2002',
      'qmspace': '\u2005',
      'bullet': '\u2022',
      'lquote': '\u2018',
      'rquote': '\u2019',
      'ldblquote': '\201C',
      'rdblquote': '\u201D',
   }
   stack = []
   ignorable = False       # Whether this group (and all inside it) are "ignorable".
   ucskip = 1              # Number of ASCII characters to skip after a unicode character.
   curskip = 0             # Number of ASCII characters left to skip
   out = []                # Output buffer.
   for match in pattern.finditer(text):

      cell,word,arg,hex,char,brace,tchar = match.groups()
      #print(cell)
      if cell:
         out.append(cell)
      if brace:
         curskip = 0
         if brace == '{':
            # Push state
            stack.append((ucskip,ignorable))
         elif brace == '}':
            # Pop state
            ucskip,ignorable = stack.pop()
      elif char: # \x (not a letter)
         curskip = 0
         if char == '~':
            if not ignorable:
                out.append('\xA0')
         elif char in '{}\\':
            if not ignorable:
               out.append(char)
         elif char == '*':
            ignorable = True
      elif word: # \foo
         curskip = 0
         if word in destinations:
            ignorable = True
         elif ignorable:
            pass
         elif word in specialchars:
            out.append(specialchars[word])
         elif word == 'uc':
            ucskip = int(arg)
         elif word == 'u':
            c = int(arg)
            if c < 0: c += 0x10000
            if c > 127: out.append(chr(c)) #NOQA
            else: out.append(chr(c))
            curskip = ucskip
      elif hex: # \'xx
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            c = int(hex,16)
            if c > 127: out.append(chr(c)) #NOQA
            else: out.append(chr(c))
      elif tchar:
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            out.append(tchar)
   return ''.join(out)




def clean_files(data_plain_text, final_data, file):
   data_preprocessing = re.sub(r' HD\\cell', '#', data_plain_text)
   data_list = data_preprocessing.split('#')
   cleaned_list = []
   dirty_articles = 0
   #print(len(data_list))
   count = 0
   for article in data_list:
      # print('***************')
      # print(article)
      article_cleaned = re.sub(r'\n*', '', article)
      flag_missing_ = 0

      if len(article_cleaned) != article_cleaned.count(' '):
         count += 1
         #print(article_cleaned)
         '''
         print(113441414)
         print(article_cleaned)
         hd = re.findall(r"(.*?)\\cell.*", article_cleaned)
         pd = re.findall(r".* PD\\cell(.*?)\\cell.*",article_cleaned)
         sn = re.findall(r".* SN\\cell(.*?)\\cell.*",article_cleaned)
         lp = re.findall(r".* LP\\cell(.*?)\\cell.*",article_cleaned)
         td = re.findall(r".* TD\\cell(.*?)\\cell.*",article_cleaned)
         print(j[0])

         temp_article = re.sub(r'BY\\cell', '#', article_cleaned)

         temp_article = re.sub(r'PD\\cell', '#', temp_article)
         temp_article = re.sub(r'SN\\cell', '#', temp_article)
         temp_article = re.sub(r'SC\\cell', '#', temp_article)
         temp_article = re.sub(r'LP\\cell', '#', temp_article)
         temp_article = re.sub(r'TD\\cell', '#', temp_article)
         # print(temp_article)
         temp_article_split = temp_article.split('#')
         # print(f'Split is {temp_article_split}')
         article_title = temp_article_split[0]
         # article_title = re.sub(r' *','',article_title)
         article_title = re.sub(' +', ' ', article_title)
         try:
            publish_date = temp_article_split[2]
            publish_date = re.sub(' +', ' ', publish_date)
            publisher = temp_article_split[3]
            publisher = re.sub(' +', ' ', publisher)
            leading_para = temp_article_split[5]
            leading_para = re.sub(' +', ' ', leading_para)
            full_text = temp_article_split[6]
            full_text = re.sub(' +', ' ', full_text)
         except:
            dirty_articles += 1
            continue

         # print(f'Publishing date is {publish_date}')
         article_title = article_title.replace("\\cell", '').strip()
         
         if not final_data[article_title]:
            
           # re.findall(r".*PD\\cell(.*?)\\cell.*", article_cleaned)
            final_data[article_title].append(publish_date.replace("\\cell",'').strip())
            final_data[article_title].append(publisher.replace("\\cell",'').strip())
            final_data[article_title].append(leading_para.replace("\\cell",'').strip())
            final_data[article_title].append(full_text.replace("\\cell",'').strip())
            final_data[article_title].append(file)
            '''
         hd = re.findall(r"(.*?)\\cell.*", article_cleaned)
         pd = re.findall(r".* PD\\cell(.*?)\\cell.*", article_cleaned)
         sn = re.findall(r".* SN\\cell(.*?)\\cell.*", article_cleaned)
         lp = re.findall(r".* LP\\cell(.*?)\\cell.*", article_cleaned)
         td = re.findall(r".* TD\\cell(.*?)\\cell.*", article_cleaned)
         CO = re.findall(r".* CO\\cell(.*?)\\cell.*", article_cleaned)
         IN = re.findall(r".* IN\\cell(.*?)\\cell.*", article_cleaned)
         NS = re.findall(r".* NS\\cell(.*?)\\cell.*", article_cleaned)
         PUB = re.findall(r".* PUB\\cell(.*?)\\cell.*", article_cleaned)
         global index 
         index += 1
         try:
             final_data[index].append(hd[0].strip())
         except:
             final_data[index].append(np.nan)

         try:
            final_data[index].append(pd[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            final_data[index].append(sn[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            final_data[index].append(lp[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            final_data[index].append(td[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            final_data[index].append(CO[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            final_data[index].append(IN[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            final_data[index].append(NS[0].strip())
         except:
            final_data[index].append(np.nan)
         try:
            pub = PUB[0].strip()
            final_data[index].append(pub)
            if 'news' in pub.lower() and 'Ltd' in pub.lower():
                final_data[index].append('News')
            elif 'fairfax' in pub.lower():
                final_data[index].append('FairFax')
            else:
                final_data[index].append('Independent')
         except:
            final_data[index].append(np.nan)
         final_data[index].append(file)
   #print(count)
   # print(dirty_articles) #Dirty articles
   # print(len(data_list)) #Total number of articles
   global files_processed
   files_processed += 1

   if not files_processed % 10:
      print(f'Processed {files_processed} files')
   return final_data
# final_data
'''
file = 'test'
current_path = os.getcwd()
test_file = current_path+'/'+'test.rtf'
file_ptr = open(test_file)

data_rtf = file_ptr.read()
t = striprtf(data_rtf)
print(t)

final_data = defaultdict(list)
files_processed = 0

final_data = clean_files(t,final_data,file)
print(final_data)

index =final_data.keys()
columns =['title','publishing_date','publisher','leading_para','full_text','Company','Industry','Subjects','Publication','PUB_Cate','file']
df = pd.DataFrame.from_dict(final_data,orient='index',columns = columns)

print(df.head())

'''

"\nfile = 'test'\ncurrent_path = os.getcwd()\ntest_file = current_path+'/'+'test.rtf'\nfile_ptr = open(test_file)\n\ndata_rtf = file_ptr.read()\nt = striprtf(data_rtf)\nprint(t)\n\nfinal_data = defaultdict(list)\nfiles_processed = 0\n\nfinal_data = clean_files(t,final_data,file)\nprint(final_data)\n\nindex =final_data.keys()\ncolumns =['title','publishing_date','publisher','leading_para','full_text','Company','Industry','Subjects','Publication','PUB_Cate','file']\ndf = pd.DataFrame.from_dict(final_data,orient='index',columns = columns)\n\nprint(df.head())\n\n"

In [43]:
df

,title,publishing_date,publisher,leading_para,full_text,Company,Industry,Subjects,Publication,PUB_Cate,file
1,,13 March 2018,Radio New Zealand News,The Australian Banking Royal Commission has he...,The ABC's Stephanie Chalmers reports.IN AN NAB...,NaN,NaN,gcat : Political/General News | ntra : Transcr...,Radio New Zealand Limited,Independent,test


In [127]:
%%time

current_path = os.getcwd()
RCMedia_path = current_path+'/RC_Media_all_files/'
final_path = RCMedia_path
sample_folder = os.listdir(final_path)
sample_files = [file for file in sample_folder if file.endswith('rtf')] #Filtering RTF files
files_processed=0
index = 0
final_data000 = defaultdict(list)
for file in sample_files:
    test_file = final_path+'/'+file
    file_ptr = open(test_file)
    data_rtf = file_ptr.read()
    data_plain_text = striprtf(data_rtf)
    #print(data_plain_text)
    final_data000 = clean_files(data_plain_text,final_data000,file)

#final_data = clean_files(data_plain_text,final_data,file)
#print(len(final_data))

Processed 10 files
Processed 20 files
Processed 30 files
Processed 40 files
Processed 50 files
Processed 60 files
Processed 70 files
Processed 80 files
Processed 90 files
Processed 100 files
Processed 110 files
Processed 120 files
Processed 130 files
Processed 140 files
Processed 150 files
Processed 160 files
Processed 170 files
Processed 180 files


In [128]:
index =final_data.keys()
columns =['title','publishing_date','publisher','leading_para','full_text','Company','Industry','Subjects','Publication','PUB_Cate','file']
df000 = pd.DataFrame.from_dict(final_data000,orient='index',columns = columns)
df000.to_csv('df000.csv')

,title,publishing_date,publisher,leading_para,full_text,Company,Industry,Subjects,Publication,PUB_Cate,file
1,New car sales like canary in a coal mine,17 August 2018,The Age - Online,Car sales are dropping. July sales were down 8...,The fact is we love cars and we all remember o...,NaN,NaN,NaN,Fairfax Media Management Pty Limited,FairFax,Factiva-20190123-1357.rtf
2,New car sales like canary in a coal mine,17 August 2018,WAToday.com.au,Car sales are dropping. July sales were down 8...,The fact is we love cars and we all remember o...,NaN,NaN,NaN,Fairfax Media Management Pty Limited,FairFax,Factiva-20190123-1357.rtf
3,Taking on major banks isn't on Australia Post'...,16 August 2018,Australian Broadcasting Corporation News,The spotlight on banking misconduct at the roy...,Mr Holgate told the ABC's AM program that ther...,auspst : Australia Post,i81402 : Commercial Banking | ibnk : Banking/C...,ccat : Corporate/Industrial News,Australian Broadcasting Corporation,Independent,Factiva-20190123-1357.rtf
4,Banking royal commission live: superannuation...,16 August 2018,The Australian - Online,ASIC flags criminal action. The Royal Commissi...,Mr Hodge begins his ‘brief’ closing observatio...,ctb : Commonwealth Bank of Australia,i81402 : Commercial Banking | i814 : Banking |...,NaN,News Ltd.,Independent,Factiva-20190123-1357.rtf
5,Katter pays the price for backing Anning's speech,17 August 2018,Canberra Times,A backlash against federal MP Bob Katter is se...,The far north Queensland MP has been shunned b...,"cufymu : Construction, Forestry, Maritime, Min...",NaN,gpol : Domestic Politics | gvuph : Upper House...,Federal Capital Press of Australia Pty Ltd,Independent,Factiva-20190123-1357.rtf
6,AMP Super powerless to stop internal fee gouging,17 August 2018,The Australian Financial Review,The trustees of for-profit superannuation fund...,"""You can't force the related-party companies t...",austmp : AMP Ltd,ipension : Pension Funds | iinv : Investing/Se...,c411 : Management Moves | npag : Page-One Stor...,Fairfax Media Management Pty Limited,FairFax,Factiva-20190123-1357.rtf
7,IOOF failed members with raid on reserve: expert,17 August 2018,The Australian Financial Review,SuperannuationFinancial services company IOOF ...,In comments to The Australian Financial Review...,aupra : Australian Prudential Regulation Autho...,ipension : Pension Funds | i81502 : Trusts/Fun...,NaN,Fairfax Media Management Pty Limited,FairFax,Factiva-20190123-1357.rtf
8,Senator in push to ban all commissions,17 August 2018,The Australian Financial Review,ExclusiveFrustrated at inaction by banks and t...,Although up-front and trailing commissions wer...,NaN,NaN,gvuph : Upper House | gpol : Domestic Politics...,Fairfax Media Management Pty Limited,FairFax,Factiva-20190123-1357.rtf
9,Katter pays the price for backing Anning's speech,17 August 2018,The Sydney Morning Herald,A backlash against federal MP Bob Katter is se...,The far north Queensland MP has been shunned b...,"cufymu : Construction, Forestry, Maritime, Min...",NaN,gpol : Domestic Politics | gvuph : Upper House...,Fairfax Media Management Pty Limited,FairFax,Factiva-20190123-1357.rtf
10,FED:Regulators face royal commission on super,17 August 2018,Australian Associated Press - General News,"BMELBOURNE, Aug 17 AAP - The regulators face q...",The deputy chairs of the Australian Securities...,NaN,NaN,c13 : Regulation/Government Policy | ccat : Co...,Australian Associated Press Pty Ltd,Independent,Factiva-20190123-1357.rtf


In [31]:

df000=pd.read_csv('df000.csv')

In [33]:
df001 = df000.dropna(axis = 0, subset = ['publishing_date'] )
df002 = df001.drop_duplicates()
df002['publishing_date'] = pd.to_datetime(df002['publishing_date'],format='%d %B %Y') 
df003 = df002.copy()
df003.file.fillna(df003.PUB_Cate,inplace=True) 
#print(df000['PUB_Cate'].value_counts() )
df003.loc[df003['Publication'].isnull(),'PUB_Cate'] = np.nan
#print(df003.isnull().sum())
df004 = df003.copy()
df004.loc[df004['Publication']=='News Ltd.','PUB_Cate'] = 'News'
df004['index'] = range(df004.shape[0])
#print(df003.isnull().sum())
df004['PUB_Cate'].value_counts() 
df005 = df004.copy()
#data['genre'] = remove(data['genre'], ' ')
#splits = split(data['genre'], ',')
df005['Company'] = df005['Company'].str.replace('[0-9a-zA-Z]* ?: ', '')
df005['Industry'] = df005['Industry'].str.replace('[0-9a-zA-Z]* ?: ', '')
df005['Subjects'] = df005['Subjects'].str.replace('[0-9a-zA-Z]* ?: ', '')
df005.to_csv('mediaData.csv')
df005.to_pickle('mediaData.pk')
df006 = df005.dropna(axis = 0, subset = ['leading_para','full_text'] )
df006.to_csv('mediaData_clean.csv')
df006.to_pickle('mediaData_clean.pk')